In [22]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [23]:
# Set random seed for purposes of reproducibility
seed = 21

In [24]:
from keras.datasets import cifar10

In [25]:
# loading in the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [26]:
# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [27]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

In [28]:
model = Sequential()

In [29]:
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

In [30]:
model.add(Dropout(0.2))

In [31]:
model.add(BatchNormalization())

In [32]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

In [33]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [34]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [35]:
model.add(Flatten())
model.add(Dropout(0.2))

In [36]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [37]:
model.add(Dense(class_num))
model.add(Activation('softmax'))

In [38]:
epochs = 2
optimizer = 'adam'

In [39]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [40]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_5 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 dropout_5 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 activation_6 (Activation)   (None, 32, 32, 64)        0         
                                                      

In [41]:
numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Epoch 1/2


2022-02-05 07:17:00.525311: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


782/782 [==============================] - ETA: 0s - loss: 1.5007 - accuracy: 0.4656

2022-02-05 07:19:21.939596: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 122880000 exceeds 10% of free system memory.


782/782 [==============================] - 146s 185ms/step - loss: 1.5007 - accuracy: 0.4656 - val_loss: 1.2038 - val_accuracy: 0.5667
Epoch 2/2
782/782 [==============================] - 145s 185ms/step - loss: 1.0169 - accuracy: 0.6395 - val_loss: 1.1597 - val_accuracy: 0.6034


In [43]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

313/313 [==============================] - 6s 19ms/step - loss: 1.1597 - accuracy: 0.6034
Accuracy: 60.34%
